In [1]:
import torchvision
from torchvision import models

In [3]:
input_size = 28
dimension = int(64 * pow(input_size/4 - 3, 2))
dimension

1024

In [4]:
pow(input_size/4 - 3, 2)

16.0

In [5]:
64*5*5

1600

In [6]:
256*3*3

2304

In [8]:
64/4*5*5

400.0

In [10]:
from utils import *
set_seed(1)

In [11]:
from data import *
dataset, train_dataset, val_dataset, test_dataset, classes = prepare_dataset()

Means: tensor([0.1702])
STDs: tensor([0.3224])


In [20]:
# generate sentences
clip_classes = [f"a drawing of a {c}" for c in classes]
clip_classes

['a drawing of a airplane',
 'a drawing of a apple',
 'a drawing of a banana',
 'a drawing of a baseball',
 'a drawing of a bear',
 'a drawing of a bicycle',
 'a drawing of a bird',
 'a drawing of a bus',
 'a drawing of a cat',
 'a drawing of a cup',
 'a drawing of a dog',
 'a drawing of a duck',
 'a drawing of a fish',
 'a drawing of a flower',
 'a drawing of a hamburger',
 'a drawing of a house',
 'a drawing of a ice_cream',
 'a drawing of a light_bulb',
 'a drawing of a lion',
 'a drawing of a nose',
 'a drawing of a monkey',
 'a drawing of a moon',
 'a drawing of a pencil',
 'a drawing of a pig',
 'a drawing of a rabbit',
 'a drawing of a shoe',
 'a drawing of a spider',
 'a drawing of a sun',
 'a drawing of a tree',
 'a drawing of a umbrella']

In [17]:
# import custom data / utils
from data import *
from utils import *
from models.model import *
from train import *
import torch

In [14]:
# https://www.pinecone.io/learn/zero-shot-image-classification-clip/
from transformers import CLIPProcessor, CLIPModel

model_id =  "openai/clip-vit-base-patch32"

processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)

Downloading: 100%|██████████| 568/568 [00:00<00:00, 142kB/s]
Downloading: 100%|██████████| 862k/862k [00:01<00:00, 840kB/s]  
Downloading: 100%|██████████| 525k/525k [00:03<00:00, 134kB/s]  
Downloading: 100%|██████████| 2.22M/2.22M [00:07<00:00, 316kB/s] 
Downloading: 100%|██████████| 389/389 [00:00<00:00, 194kB/s]
Downloading: 100%|██████████| 4.19k/4.19k [00:00<00:00, 1.40MB/s]
Downloading: 100%|██████████| 605M/605M [00:36<00:00, 16.6MB/s] 


In [18]:
# DEVICE = check_device()

In [21]:
# create label tokens
classes_tokens = processor(
    text=clip_classes,
    padding=True,
    images=None,
    return_tensors='pt'
)

classes_tokens['input_ids'][0][:10]

tensor([49406,   320,  3610,   539,   320, 16451, 49407, 49407, 49407])

In [22]:
# encode tokens to sentence embeddings
classes_emb = model.get_text_features(**classes_tokens)
# detach from pytorch gradient computation
label_emb = classes_emb.detach().numpy()
label_emb.shape


(30, 512)

In [25]:
label_emb.min(), label_emb.max()

(-1.2360154, 5.4198923)

In [26]:
# normalization
label_emb = label_emb / np.linalg.norm(label_emb, axis=0)
label_emb.min(), label_emb.max()

(-0.6383198, 0.6149943)

In [29]:
from torch.utils.data import DataLoader
BATCH_SIZE = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [66]:
data_iter = iter(train_loader)
images, labels = next(data_iter)

In [68]:
from PIL import Image

In [71]:
image = processor(
  text=None,
  images=Image.fromarray(images[0].numpy()),
  return_tensors='pt'
)['pixel_values']

TypeError: Cannot handle this data type: (1, 1, 28), <f4

In [65]:
img_emb = model.get_image_features(images[0])

RuntimeError: Given groups=1, weight of size [768, 3, 32, 32], expected input[1, 1, 28, 28] to have 3 channels, but got 1 channels instead